In [ ]:
!pip install transformers==3.5.0
!pip install pytorch-lightning
!pip install ftfy
!pip install neptune-client==0.4.130
!pip install efficientnet-pytorch 

In [ ]:
import torch
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu" 
print(dev, torch.cuda.get_device_name(0))
device = torch.device(dev)

cuda:0 Tesla V100-SXM2-16GB


In [ ]:
import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
import torchvision
from torchvision import transforms, utils
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer
import neptune
from pytorch_lightning.loggers import NeptuneLogger
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import NeptuneLogger

from random import randrange
import random
import numpy as np
import collections
import os
import glob
import json
from ftfy import fix_encoding

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageChops
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import imageio
from typing import Callable, Dict, List, Tuple

### Métricas

In [ ]:
def normalize_answer(s):
    """Lower text and remove extra whitespace."""

    def white_space_fix(text):
        return ' '.join(text.split())

    def lower(text):
        return text.lower()

    return white_space_fix(lower(s))

def get_tokens(s):
    if not s: return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

### DocVQA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tarfile
### unzip files
for mode in ['train','val','test']:
    tf = tarfile.open("/content/drive/MyDrive/OCR_checkpoints/{}.tar.gz".format(mode))
    tf.extractall()

In [ ]:
class DocVQA(Dataset): 
    '''
    Adaptado a partir da classe dataset implementada pelo Diedre
    '''

    def __init__(self,
                 mode: str,
                 tokenizer_string: str = 't5-base', #'microsoft/layoutlm-base-uncased',
                 transform: object = None,
                 seq_len: int = 512,
                 no_image: bool = False):
        '''
        mode: one of train, val and test.
        tokenizer_string: input tokenizer string 
        transform: transforms to be applied to the document image if applicable.
        seq_len: maximum sequence len of encoded tokens.
        no_image: if True, don't load document images.
        returns:
            dict:
                document: transformed document image.
                input_tokens: tokenized text contained in the document.
                input_text: text contained in the document.
                bboxes: bounding boxes for each OCR detection in the document, on the format [tl_col, tl_row, br_col, br_row].
        '''
        super().__init__()
        assert mode in ["train", "val", "test"]
        with open(f"{mode}/{mode}_v1.0.json", 'r') as data_json_file:
            self.data_json = json.load(data_json_file)

        self.folder = f"{mode}"
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_string)
        self.transform = transform
        self.seq_len = seq_len
        self.mode = mode
        self.no_image = no_image

        print(f"{self.mode} DocVQA folder {self.folder} tokenizer {self.tokenizer} seq_len {self.seq_len} "
              f"no_image {self.no_image}")

      

    def __len__(self):
        return len(self.data_json["data"])

    def __getitem__(self, i: int):
        data = self.data_json["data"][i]

        if self.no_image:
            document = "NA"
        else:
            document = np.array(Image.open(os.path.join(self.folder, data["image"])).convert('RGB'))
            
        ID = data["ucsf_document_id"] + '_' + data["ucsf_document_page_no"]
        ocr_file = os.path.join(self.folder, "ocr_results", ID + ".json")
        with open(ocr_file, 'r') as ocr_file:
            ocr_info = json.load(ocr_file)

      # Retira o texto da imagem a partir do OCR
        lines = ocr_info['recognitionResults'][0]['lines']
        nlines = len(lines)

        bboxes = []
        input_text = ''
        for line in range(nlines):
            input_text += lines[line]['text'] + ' '
           

        question = self.data_json["data"][i]['question']
        input_context = 'Question: ' + question + ' Context: ' + input_text #+ ' </s>'
        input_tokens = self.tokenizer.encode_plus(input_context, padding='max_length', truncation=True, max_length=self.seq_len, return_tensors='pt',return_token_type_ids=True)
      
        target_text = random.choice(data["answers"]) if self.mode == "train" else data.get("answers", ["NA"])[0]
        target = self.tokenizer.encode(target_text, padding='max_length', truncation=True, max_length=32, return_tensors='pt')[0]
      
        if self.transform is not None:
            document = self.transform(document)

        return_dict = {"document": document,
                       "input_ids": input_tokens["input_ids"].squeeze(),
                       "token_type_ids": input_tokens["token_type_ids"].squeeze(),
                       "attention_mask": input_tokens["attention_mask"].squeeze(),
                       "input_context": input_context,
                       "target": target,
                       "target_text": target_text}

        return return_dict

In [ ]:
test_augmentations = transforms.Compose([
     transforms.ToPILImage(),
     transforms.Resize((740, 560)),
     transforms.ToTensor(),
     ])

tokenizer_string = "/content/drive/MyDrive/OCR_checkpoints/T5_NQ"
seq_len = 512

### Modelo

In [ ]:
class T5Finetuner(pl.LightningModule):


    def __init__(self, train_dataloader, val_dataloader, test_dataloader, params):
        super(T5Finetuner, self).__init__()

        self.params = params
        
        self._train_dataloader = train_dataloader
        self._val_dataloader   = val_dataloader
        self._test_dataloader  = test_dataloader

        self.encoder = EfficientNet.from_pretrained(model_name='efficientnet-b4', advprop= params['advprop'])
        self.decoder = T5ForConditionalGeneration.from_pretrained(tokenizer_string)

        self.cnn1 = nn.Conv2d(in_channels=160, out_channels=self.decoder.config.d_model, kernel_size=1) # b0 => in_channels = 112
                                                                                                        # b4 => in_channels = 160
        self.tokenizer = T5Tokenizer.from_pretrained(tokenizer_string)                                  # b7 => in_channels = 224
        self.learning_rate = params['learning_rate']

    def generate(self, embeddings):
        ''' 
        Recebe as features do encoder e retorna dos decoded ids.
        Adaptado da tarefa sobre decodificadores do semestre passado 
        '''
        max_length = self.params['seq_len']

        # Add start of sequence token
        decoded_ids = torch.full((embeddings.shape[0], 1),
                                 self.decoder.config.decoder_start_token_id,
                                 dtype=torch.long).to(embeddings.device)

        encoder_hidden_states = self.decoder.get_encoder()(inputs_embeds=embeddings)

        for step in range(max_length-1): # testar max_length
            logits = self.decoder(decoder_input_ids=decoded_ids, encoder_outputs=encoder_hidden_states)[0]
            next_token_logits = logits[:, -1, :]

            # Greedy decoding
            next_token_id = next_token_logits.argmax(1).unsqueeze(-1)
            
            # Check if output is end of senquence for all batches
            if torch.eq(next_token_id[:, -1], self.tokenizer.eos_token_id).all():
                break

            # Concatenate past ids with new id, keeping batch dimension
            decoded_ids = torch.cat([decoded_ids, next_token_id], dim=-1)

        return decoded_ids

    def forward(self, batch): 
        
        #image features
        features = model.encoder.extract_endpoints(batch['document'])["reduction_4"]  # Shape (N, 112, 16, 16)
        features = self.cnn1(features)
        embeddings = features.permute(0, 2, 3, 1).reshape(features.shape[0], -1, self.decoder.config.d_model)

        # text features
        question_embeds = self.decoder.encoder(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], return_dict=True).last_hidden_state

        # concatenate image and question sentence embeddings
        inputs_embeds = torch.cat((question_embeds, embeddings), dim=1)
        target_ids = batch['target']

        if self.training:
            outputs = self.decoder(inputs_embeds=inputs_embeds, labels=target_ids, return_dict=True)
            return outputs.loss
        else:
            return self.generate(inputs_embeds)

    def training_step(self, batch, batch_idx): 
        loss = self(batch)
        self.log('loss', loss, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        pred_tokens = self(batch)
        decoded_pred = [fix_encoding(self.tokenizer.decode(tokens)) for tokens in pred_tokens]
        return {"pred": decoded_pred, "target": batch["target_text"]}

    def test_step(self, batch, batch_idx):
        pred_tokens = self(batch)
        decoded_pred = [fix_encoding(self.tokenizer.decode(tokens)) for tokens in pred_tokens]
        return {"pred": decoded_pred, "target": batch["target_text"]}

    def validation_epoch_end(self, outputs):
        # Flatten dos targets e preds para arrays
        trues = sum([list(x['target']) for x in outputs], [])
        preds = sum([list(x['pred']) for x in outputs], [])

        n = randrange(len(trues))
        print(f"\nSample Target: {trues[n]}\nPrediction: {preds[n]}\n")

        f1 = []
        exact = []
        for true, pred in zip(trues, preds):
            f1.append(compute_f1(a_gold=true, a_pred=pred))
            exact.append(compute_exact(a_gold=true, a_pred=pred))
        f1 = np.mean(f1)
        exact = np.mean(exact)

        self.log("val_f1", f1, prog_bar=True)
        self.log("val_exact", exact, prog_bar=True)

    def test_epoch_end(self, outputs):
        # Flatten dos targets e preds para arrays
        trues = sum([list(x['target']) for x in outputs], [])
        preds = sum([list(x['pred']) for x in outputs], [])  # TESTAR FIX ENCODING

        for random in range(5):
            n = randrange(len(trues))
            print(f"\nSample Target: {trues[n]}\nPrediction: {preds[n]}\n")

        f1 = []
        exact = []
        for true, pred in zip(trues, preds):
            f1.append(compute_f1(a_gold=true, a_pred=pred))
            exact.append(compute_exact(a_gold=true, a_pred=pred))
        f1 = np.mean(f1)
        exact = np.mean(exact)

        self.log("test_f1", f1, prog_bar=True)
        self.log("test_exact", exact, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(
            [p for p in self.parameters() if p.requires_grad],
            lr=self.learning_rate, eps=1e-08)

    def train_dataloader(self):
        return self._train_dataloader

    def val_dataloader(self):
        return self._val_dataloader

    def test_dataloader(self):
        return self._test_dataloader

    


In [ ]:
learning_rate =  5e-5#@param {type: "number"} 5e-5
batch_size =    1#@param {type: "integer"} 32
sequence =  32#@param {type: "integer"}
patience =  5#@param {type: "integer"}
max_epochs =  5#@param {type: "integer"}
advprop = False#@param {type: "boolean"}

params = {  
    'advprop': advprop,  
    'batch_size': batch_size,
    'seq_len': sequence,
    'learning_rate': learning_rate,
    'max_epochs': max_epochs,
    'patience': patience,
    'monitor_variable': 'val_f1'
}

In [ ]:
data_augmentations = transforms.Compose([
     transforms.ToPILImage(),
     transforms.Resize((600, 400)), # 740, 560
     transforms.ToTensor(),
     ])

train = DocVQA('train', tokenizer_string, data_augmentations, seq_len)
val = DocVQA('val', tokenizer_string, data_augmentations, seq_len)
test = DocVQA('val', tokenizer_string, data_augmentations, seq_len)

train_loader = DataLoader(train,
                          shuffle = True,
                          batch_size=params['batch_size'],
                          num_workers=4)

val_loader = DataLoader(val,
                        batch_size=params['batch_size'],
                        num_workers=4)


test_loader = DataLoader(test,
                         batch_size=params['batch_size'],
                         num_workers=4)

print('Seq_len:', seq_len)

train DocVQA folder train tokenizer PreTrainedTokenizer(name_or_path='/content/drive/MyDrive/OCR_checkpoints/T5_NQ', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_

### Treinamento

In [ ]:
#### DocVQA
#checkpoint_path = '/content/drive/MyDrive/OCR_checkpoints/DocVQA-EffT5-epoch=11-val_f1=0.58-val_exact=0.48.ckpt' # 2  
checkpoint_path = '/content/drive/MyDrive/OCR_checkpoints/DocVQA-EffT5-epoch=13-val_f1=0.58-val_exact=0.48.ckpt' # 4  


checkpoint_dir = os.path.dirname(os.path.abspath(checkpoint_path))
print(f'Files in {checkpoint_dir}: {os.listdir(checkpoint_dir)}')
print(f'Saving checkpoints to {checkpoint_dir}')


checkpoint_callback = ModelCheckpoint(prefix="DocVQA-EffT5",
                                      filepath="/content/drive/MyDrive/OCR_checkpoints/{epoch}-{val_f1:.2f}-{val_exact:.2f}",
                                      save_top_k=-1)  # -1 = Keeps all checkpoints, 1 = save best

resume_from_checkpoint = None
if os.path.exists(checkpoint_path):
    print(f'Restoring checkpoint: {checkpoint_path}')
    resume_from_checkpoint = checkpoint_path

callbacks = [pl.callbacks.EarlyStopping(monitor=params['monitor_variable'], 
                                        patience=params["patience"], 
                                        mode='max')]

# Log results to Neptune.
neptune_logger = pl.loggers.neptune.NeptuneLogger(
    api_key='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNmJkMmRkNGMtMzNlMi00MzY2LThiZTUtODMxYmFlMzUwYzcwIn0=',
    project_name='guilhermemr04/sandbox',
    params=params,)


trainer = pl.Trainer(gpus=1,
                     logger=neptune_logger,
                     precision=32, 
                     log_gpu_memory=True,
                     max_epochs=17,#params['max_epochs'],
                     check_val_every_n_epoch=1,
                     profiler=True,
                     callbacks=None,#callbacks,
                     accumulate_grad_batches=32,
                     checkpoint_callback= checkpoint_callback,
                     val_check_interval=0.25,  # check val when 25% is trained
                     limit_val_batches=0.05,  # use 5% of dev set to evaluate
                     progress_bar_refresh_rate=100,
                     resume_from_checkpoint=resume_from_checkpoint)
                     

model = T5Finetuner(train_dataloader=train_loader,
                    val_dataloader=val_loader,
                    test_dataloader=test_loader,
                    params=params)

Files in /content/drive/MyDrive/OCR_checkpoints: ['img2text-basic-epoch=0-val_exact=0.59.ckpt', 'img2text-basic-epoch=0-val_exact=0.70.ckpt', 'img2text-intermediate-epoch=8-val_exact=0.36.ckpt', 'img2text-final-epoch=47-val_exact=0.37.ckpt', 'img2text-final-epoch=74-val_exact=0.44.ckpt', 'img2text-Nota_Fiscal-epoch=88-val_exact=0.27.ckpt', 'img2text-Nota_Fiscal-epoch=95-val_exact=0.27.ckpt', 'img2text-teste-epoch=13-val_exact=0.33.ckpt', 'img2text-teste_2-epoch=19-val_exact=0.25.ckpt', 'img2text-teste_2-epoch=21-val_exact=0.28.ckpt', 'img2text-teste_2-epoch=22-val_exact=0.30.ckpt', 'img2text-teste_3-epoch=31-val_exact=0.10.ckpt', 'img2text-teste_3-epoch=32-val_exact=0.09.ckpt', 'img2text-SROIE-epoch=43-val_exact=0.22.ckpt', 'img2text-SROIE-epoch=51-val_exact=0.22.ckpt', 'img2text-SROIE_v2-epoch=40-val_exact=0.43.ckpt', 'img2text-SROIE_v2-epoch=47-val_exact=0.42.ckpt', 'img2text-SROIE_v3-epoch=41-val_exact=0.44.ckpt', 'img2text-SROIE_v3-epoch=46-val_exact=0.46.ckpt', 'test.tar.gz', 'val

NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loaded pretrained weights for efficientnet-b4


In [ ]:
trainer.fit(model)


  | Name    | Type                       | Params
-------------------------------------------------------
0 | encoder | EfficientNet               | 19.3 M
1 | decoder | T5ForConditionalGeneration | 222 M 
2 | cnn1    | Conv2d                     | 123 K 
-------------------------------------------------------
242 M     Trainable params
0         Non-trainable params
242 M     Total params
Restored states from the checkpoint file at /content/drive/MyDrive/OCR_checkpoints/DocVQA-EffT5-epoch=13-val_f1=0.58-val_exact=0.48.ckpt



Sample Target: university of california
Prediction: university of california



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


KeyboardInterrupt: ignored

In [ ]:
trainer.test(model) #2


Sample Target: $3,150.00
Prediction: $3,150.00


Sample Target: $1,384
Prediction: $1.384.


Sample Target: COPY
Prediction: RJ REYNOLDS TOBACCO COMPANY


Sample Target: indicate your approval, or criticism and return to us promptly
Prediction: Minutes of yesterday's Committee Meeting


Sample Target: Joe
Prediction: Joe Copied/Aff


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_exact': 0.45971209571882593, 'test_f1': 0.5620391020905482}
--------------------------------------------------------------------------------


[{'test_exact': 0.45971209571882593, 'test_f1': 0.5620391020905482}]

In [ ]:
trainer.test(model) # 4


Sample Target: fao
Prediction: food and agriculture organization


Sample Target: Department of Obstetrics and Gynecology
Prediction: Department of Obstetrics and Gynecology


Sample Target: 1010
Prediction: 1010


Sample Target: Address
Prediction: Address


Sample Target: center support
Prediction: Center Support


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_exact': 0.4733595064498037, 'test_f1': 0.5720039645077956}
--------------------------------------------------------------------------------


[{'test_exact': 0.4733595064498037, 'test_f1': 0.5720039645077956}]